In [3]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

# Pre Exercise

In the google doc sheet we have 41 restaurant and 44 people. There are about 1000 rating in total. We want to do collaborative filtering with 7 hidden features.

https://docs.google.com/spreadsheets/d/143AcuFLKRLkz7yoKiEas0rABfuKPYCSlVKY1DUbFiBo/edit#gid=0

1) What is the size of matrix R?

In [1]:
# 41

2) What is the size of matrix P?

In [1]:
# 44

3) What should be the dimension of the rating matrix T? Write a numpy statement to calculate the $T$ from $R$ and $P$.

In [1]:
# 41 x 44

4)Import the google sheet. (export it to csv and read line by line). The google sheet and the notes has a bit different conventtion for rating matrix so be careful.

In [4]:
def read_rating():
    with open('rating.csv') as f:
        iline = 0
        lines = f.readlines()
        useful_lines = lines[3:]

        names = lines[2].split(',')[2:]
        names = map(lambda x: x.strip(), names)

        all_ratings = []
        all_defined = []
        rnames = []
        for iline, line in enumerate(useful_lines):
            tokens = line.split(',')
            tokens = map(lambda x: x.strip(), tokens)
            rname = tokens[1]
            ratings = tokens[2:]
            defined = map(lambda x: 0 if x=='' or x=='"' else 1, ratings )
            def clean_cast(x):
                # print x
                return 0 if x=='' or x=='"' else float(x)
            ratings = map(lambda x: clean_cast(x), ratings)
            all_ratings.append(ratings)
            all_defined.append(defined)
            rnames.append(rname)
        #print all_ratings, all_defined , rnames
        T = np.array(all_ratings)
        H = np.array(all_defined)
    return T, H, names, rnames
T, H, names, rnames = read_rating()

5) What is the dimension of rating matrix? AFTER you figure out, check it with rating.shape.

In [5]:
T.shape

(41L, 44L)

6) What is the dimension of has_rating matrix? AFTER you figure out, check it with rating.shape.

In [6]:
H.shape

(41L, 44L)

7)The update rule for $P$ is


$P = P - \eta\; dP$

Write down the formula for $dP$. Then, write a numpy code to calculate $dP$.

$P_{new} = P_{old} + 2\lambda R_{old} (H \otimes (\tilde{T} - R^T_{old}P_{old}))$

8)The update rule for $R$ is

$R = R- \eta\; dR$

Write down the formula for dR. Then, write a numpy code to calculate dR.

$R_{new} = R_{old} + 2\lambda P_{old} (H \otimes (\tilde{T} - R^T_{old}P_{old}))^T$

9) Write the numpy code to calculate the cost. (you may find np.sum useful)

# Exercise
You have given CSV file for all the rating. Now your job is to figure out what is the next restaurant you should go next using collaborative filtering with gradient descent.

The idea is to keep update matrix F and P with the update rule we find above many many times.

Debugging this is quite a pain. One way to do it is to monitor the value of the cost function making sure it is going in the right direction.

If you are doing it right this should not be more than 50 lines.

The convergence is quite slow. So try it 10k + times.

In [34]:
npeople = len(names)
nrest = len(rnames)
nfeature = 7
np.random.seed(17)
P = np.random.randn(nfeature, npeople)
R = np.random.randn(nfeature, nrest)
print P.shape
print R.shape

(7L, 44L)
(7L, 41L)


In [35]:
def guess(R, P):
    return np.dot(R.T, P)

def score(R, P):
    return np.sum(H*(T - guess(R,P))**2) # cost function

print score(R,P)

l = 0.005
def find_R_P(R, P):
    for i in range(100000):
        HT = (H * (T - np.dot(R.T, P)))
        R_old = R.copy()
        R = R + l * np.dot(P, HT.T)
        P = P + l * np.dot(R_old, HT)
        if (i%5000==0):
            print i, score(R,P)
    return R, P

R,P = find_R_P(R,P)

15693.0280721
0 11992.3065613
5000 76.1232706684
10000 73.6413859765
15000 72.9864280552
20000 72.7348353729
25000 72.5672594731
30000 72.4002689961
35000 72.2269669945
40000 72.0643140757
45000 71.9232788579
50000 71.8053200293
55000 71.7025221284
60000 71.5999504693
65000 71.4864917429
70000 71.3695644389
75000 71.2617513429
80000 71.1648661595
85000 71.076025774
90000 70.9927584886
95000 70.9139828158


# Debugging Tip
To compare your guess rating(G*has_rating) and the actual rating graphically use

plt.matshow(rating, vmin=0, vmax=5)

 if they look almost the same then your algorithm is doing the right thing 

Once you have F and P and guess matrix.
I wrote this so that visualizing your result is easier.

In [29]:
from IPython.display import HTML

class TableCell:
    
    def __init__(self, text, tc=None, color=None):
        self.text = text
        self.tc = tc
        self.color = color
    
    def to_html(self):
        return '<td>%s</td>'%self.text

#the rating and guess matrix has different convention from the notes so be sure to transpose it first
def maketable(rating, has_rating, guess, restaurants, names):
    n_rests = len(restaurants)
    n_names = len(names)
    tab = np.empty((n_rests+1, n_names+1),dtype='object')
    #print tab.shape

    for irest in range(n_rests):
        tab[irest+1,0] = restaurants[irest]

    for iname in range(n_names):
        tab[0,iname+1] = names[iname]

    for irest in range(n_rests):
        for iname in range(n_names):
            if not has_rating[iname, irest]:
                tab[irest+1, iname+1] = TableCell('<span style="color:red">%3.2f</span>'%(guess[iname, irest]))
            else:
                tab[irest+1, iname+1] = TableCell('<span style="color:blue">%3.2f</span><span style="color:red">(%3.2f)</span>'%(rating[iname, irest], guess[iname, irest]))
    #now convert tab array to nice html table
    nrow, ncol = tab.shape
    t = []
    t.append('<table>')
    for irow in range(nrow):
        t.append('<tr>')
        for icol in range(ncol):
            cell = tab[irow,icol]
            if cell is not None:
                if isinstance(cell,TableCell):
                    t.append(tab[irow, icol].to_html())
                else:
                    t.append('<td>')
                    t.append(tab[irow, icol])
                    t.append('</td>')
            else:
                t.append('<td></td>')
        t.append('</tr>')  
    t.append('</table>')
    return '\n'.join(t)

In [32]:
# call it like this

HTML(maketable(T.T, H.T, np.dot(R.T,P).T, rnames, names))

,Piti,Meena,Pitoon,Sohum,Sam,Keng,O,Yok,Kitty,Time,Robroo,Peem,Chanon,Ohm,Sun,Tii,Tow,John,Mint,Opal,Kelly,Jay,Sharik,Ou,PJ,Punawit,Martin,Ploy,Majeed,Bossy,Sea,Pan,Ice,Karn,May,Rin,Peter,Ham,Benz,Billy,Kanat,Sam,Boss,Best
Mai-tok-mai-tak,3.27,4.75,2.00(1.95),6.62,6.61,2.00(2.05),2.00(2.33),3.00(2.92),4.91,3.00(3.24),3.00(3.02),5.00(4.34),2.23,3.00(2.96),2.00(2.07),3.00(3.08),3.00(3.38),0.15,2.00(1.94),3.00(2.98),3.00(3.16),3.00(2.56),3.57,2.00(2.28),2.66,3.24,2.62,4.00(2.88),3.84,3.00(3.04),4.00(4.07),-4.43,3.77,3.00(3.53),4.00(4.09),5.50,6.64,1.48,4.00(3.94),4.00(4.48),4.00(3.57),4.50,4.00(3.90),5.00(5.14)
Puttharaksa,4.00(3.44),2.68,4.00(3.72),4.00(3.67),4.00(4.20),3.00(3.40),4.00(4.19),3.00(3.14),3.19,3.00(3.08),4.79,4.00(4.31),3.00(2.67),4.00(4.19),3.00(3.31),4.00(4.05),4.00(3.67),3.00(2.19),3.00(3.57),3.00(3.39),3.00(2.82),3.00(3.32),3.00(2.98),3.00(2.99),3.95,3.18,3.44,3.00(2.94),3.00(3.14),3.61,4.00(4.09),4.00(3.98),4.00(4.09),3.10,4.00(3.32),3.00(3.01),3.06,3.20,1.00(2.05),4.00(3.65),3.00(2.90),5.38,4.00(3.68),4.00(3.79)
Big Mamma,0.97,4.00(4.00),7.18,5.00(4.95),5.00(5.02),1.28,-0.23,1.48,4.00(4.02),2.67,-1.06,2.63,3.44,6.99,2.54,0.36,2.70,-1.57,4.94,4.39,7.06,2.84,5.31,0.53,-0.21,-0.73,2.55,3.08,3.12,0.47,3.13,13.46,5.30,2.28,4.00(4.03),4.00(4.00),0.30,3.00(2.98),3.19,5.77,1.06,2.60,4.53,7.54
Seefah,4.28,2.00(2.03),4.00(3.90),4.00(4.18),5.00(4.71),4.23,4.00(3.60),3.00(3.88),4.06,5.00(3.84),2.10,4.00(3.80),3.00(3.45),4.95,4.14,4.00(4.08),4.73,3.00(3.42),5.23,4.10,4.73,3.73,4.00(4.05),2.00(2.52),4.62,4.00(3.56),3.72,3.00(3.44),3.81,2.00(2.24),3.99,18.44,5.23,3.35,3.00(3.35),3.28,3.00(3.04),4.00(3.71),3.26,4.00(4.14),3.41,4.03,5.00(4.36),4.00(4.23)
Music Square,2.00(1.99),-11.14,-0.67,5.50,3.00(3.05),4.00(3.83),2.36,4.00(3.34),4.00(4.10),4.00(4.09),4.00(3.99),3.65,2.01,-1.83,4.00(4.11),4.00(3.99),5.00(5.13),2.86,5.00(5.27),3.23,4.00(3.65),3.00(3.23),4.24,2.00(1.87),8.71,5.00(5.02),1.68,3.00(3.04),3.85,4.21,4.00(4.08),32.17,6.13,4.00(4.01),5.00(4.53),4.00(4.02),11.36,1.89,2.74,2.00(2.35),4.00(4.31),5.58,4.00(3.93),3.00(2.96)
Mamma Mia,3.00(3.37),4.00(3.93),4.80,5.00(5.23),5.00(5.03),3.13,4.00(4.14),4.00(3.21),4.43,4.00(3.56),5.00(4.97),5.00(5.26),3.00(3.14),5.00(4.97),3.00(3.31),5.00(4.89),4.00(3.75),0.89,4.00(3.88),4.00(4.00),4.00(3.82),4.00(3.71),3.96,3.00(2.96),3.70,3.00(3.12),4.00(3.77),3.00(3.51),4.03,4.00(3.98),5.00(4.83),3.00(3.02),5.00(5.01),4.00(3.76),5.00(4.60),5.00(4.96),4.76,3.00(3.21),3.00(2.93),5.00(5.10),3.00(3.23),6.58,4.00(4.67),5.91
Srijan,4.00(3.96),11.43,3.00(2.74),0.36,-1.14,3.00(3.63),7.20,5.00(3.82),3.58,3.00(2.80),2.00(2.46),4.00(3.87),2.94,2.00(1.95),2.00(2.07),2.00(1.98),2.00(2.21),2.55,1.00(1.25),1.91,0.27,3.31,1.28,3.00(2.83),1.73,4.00(3.96),4.49,2.98,3.00(3.56),2.00(1.59),2.68,4.00(3.97),2.14,3.09,3.00(2.46),5.99,1.51,3.00(3.21),3.00(3.29),4.00(4.40),3.00(3.16),5.36,4.00(3.47),3.00(3.27)
Steak House,-5.75,3.00(3.00),-10.09,4.00(4.07),4.00(3.99),2.76,-0.19,4.77,4.00(4.00),3.42,-5.01,-4.35,0.84,-19.44,0.79,-49.88,6.51,1.00(1.03),-4.74,0.22,0.00(-0.07),1.99,5.00(4.98),3.00(2.85),-2.64,3.00(2.98),0.30,3.71,0.78,1.37,-0.15,3.00(3.00),-0.03,4.26,2.08,6.16,3.00(3.01),2.00(2.15),7.89,3.30,5.50,-2.73,1.81,-2.78
Anya,1.00(1.05),4.00(3.97),4.00(3.90),4.00(3.96),5.00(4.89),4.00(4.26),4.00(3.99),4.00(4.13),4.00(3.81),4.00(3.94),3.00(3.17),3.00(2.95),4.00(3.65),1.00(1.06),4.00(4.11),-11.20,5.46,2.00(2.00),4.00(3.66),4.00(4.38),4.00(4.26),5.00(4.43),5.00(5.14),4.00(3.72),2.22,2.50,4.00(4.02),4.00(4.33),3.00(3.09),4.00(3.78),4.17,9.00,5.00(4.89),4.00,4.10,3.91,0.56,5.00(5.07),4.00(3.63),5.00(5.38),3.00(3.47),4.82,5.00(4.68),4.00(4.14)
Chicky Chic,2.00(1.88),3.00(3.13),2.00(2.35),2.00(2.09),0.00(-0.05),2.00(2.17),5.00(4.86),2.00(2.23),3.00(2.96),2.00(2.27),5.00(4.86),4.00(4.18),2.00(1.86),1.00(0.94),2.00(1.75),4.11,1.71,0.00(0.32),2.00(1.56),2.00(1.88),0.00(0.54),2.00(2.64),2.00(1.68),3.00(2.22),3.00(3.00),3.00(2.98),3.00(2.82),2.00(2.35),2.98,3.00(3.37),3.00(3.15),1.94,3.00(2.83),3.00(2.87),3.00(3.43),5.00(5